In [1]:
import cv2
import slideio
import numpy as np
from pathlib import Path
import os

class SlideReader:

    def __init__(self, input_slide):
        slide = slideio.open_slide(input_slide, 'SVS')
        self._num_scenes = slide.num_scenes
        self._scene = slide.get_scene(0)
        self._set_tile_resolution()

    NETWORK_TILE_WIDTH = 256
    NETWORK_TILE_HEIGHT = 256

    ####################################################################################################################
    # Setting of the tile size depending on WSI magnification
    ####################################################################################################################
    # default tile size for magnification equal to 5x
    TILE_WIDTH = 128
    TILE_HEIGHT = 128
    def _set_tile_resolution(self):
        if self._get_magnification() == 40:
            self._tile_width = 8 * self.TILE_WIDTH
            self._tile_height = 8 * self.TILE_HEIGHT
        elif self._get_magnification() == 20:
            self._tile_width = 4 * self.TILE_WIDTH
            self._tile_height = 4 * self.TILE_HEIGHT
        else:
            raise Exception('I do not know how to work with this magnification %d', self._get_magnification())

    def _get_magnification(self):
        return self._scene.magnification

    def _get_resolution(self):
        return self._scene.size

    def _get_scenes(self):
        return self._num_scenes

    ####################################################################################################################
    # BEGIN: Na podstawie maski pobranie danych z obrazów SVS i zapisanie ich w formie kafli
    ####################################################################################################################
    def prepare_tiles(self, output_tile):

        def _get_coords(width, height, tile_width, tile_height):
            x_coords = range(0, width, tile_width)
            y_coords = range(0, height, tile_height)
            return x_coords, y_coords

        def _mirror(mask, final_height, final_width):

            if mask.shape[0] < final_height:
                mask_org = mask
                mask_flip = np.flip(mask, axis=0)
                mask = np.concatenate((mask_org, mask_flip), axis=0)
                if mask.shape[0] > final_height:
                    mask = mask[:final_height, :]

            if mask.shape[1] < final_width:
                mask_org = mask
                mask_flip = np.flip(mask, axis=1)
                mask = np.concatenate((mask_org, mask_flip), axis=1)
                if mask.shape[1] > final_width:
                    mask = mask[:, :final_width]

            if mask.shape[0] < final_height or mask.shape[1] < final_width:
                mask = _mirror(mask, final_height, final_width)

            return mask

        def _mirror_pixels(pixels, start_x, start_y):
            start_x = int(np.floor(start_x * self.NETWORK_TILE_WIDTH))
            start_y = int(np.floor(start_y * self.NETWORK_TILE_HEIGHT))

            if start_x == 0:
                start_x = self.NETWORK_TILE_WIDTH
            if start_y == 0:
                start_y = self.NETWORK_TILE_HEIGHT

            pixels = pixels[:start_y, :start_x]

            return _mirror(pixels, self.NETWORK_TILE_HEIGHT, self.NETWORK_TILE_WIDTH)


        # find coordinates of all slides in the image
        x_coords, y_coords = _get_coords(self._scene.size[1], self._scene.size[0],
                                         self._tile_width, self._tile_height)

        # iterating over tiles
        for idx_x, x_img in enumerate(x_coords):
            for idx_y, y_img in enumerate(y_coords):
                    pixels = self._scene.read_block(rect=(y_img, x_img,
                                                          self._tile_height, self._tile_width),
                                                    size=(self.NETWORK_TILE_HEIGHT,
                                                          self.NETWORK_TILE_WIDTH)
                                                    )

                    if pixels.shape[0] < self.NETWORK_TILE_HEIGHT or pixels.shape[1] < self.NETWORK_TILE_WIDTH:
                        pixels = _mirror(pixels, self.NETWORK_TILE_HEIGHT, self.NETWORK_TILE_WIDTH)

                    cv2.imwrite(output_tile + '_' + str(idx_x).zfill(3) + '_' + str(idx_y).zfill(3) + '.png', pixels)


if __name__ == '__main__':

    # The SlideReader works in a following manner:
    # It takes the SVS whole slide scan and reads its info - the resolution and magnification are the most important ones.
    # It works only with magnification 20x and 40x see -> _set_tile_resolution
    # It assumes that we want to have a tile of 128x128 pixels resolution at magnification 5x and recalculates it for
    # the ones that are useful. The number should be changed according to your needs.
    # Then in prepare_tiles it takes the output where to store the tiles and goes over the slide.
    # In the case the last column or row are smaller than the demanded tile size, they are mirrored.
    # Finally, the tile is scaled to the NETWORK resolution - also set according to demands.

     # Process current directory and subdirectories
    #process_folder(os.getcwd())

    #point to the folder containing wsi - INPUT
    #folder_name = 'wsi_svs'
    #folder_path = os.path.join(os.getcwd(), folder_name)
    folder_path = 'Z:/Admin/KAPONGO/0 MSc/WSI for Testing/WSI - svs'
    
    # loop through the ndpi in the folder...
    for file_path in Path(folder_path).glob('*.svs'):

        slide_reader = SlideReader(str(file_path))

        output_path = Path(os.getcwd()).joinpath(file_path.stem)
        output_path.mkdir(exist_ok=True)

        slide_reader.prepare_tiles(str(output_path) + "/" + file_path.stem)
################
        
######## Reads images from the working directory ############################
    #for file_path in Path(os.getcwd()).glob('*.svs'):

       # slide_reader = SlideReader(str(file_path))

        #output_path = Path(os.getcwd()).joinpath(file_path.stem)
        #output_path.mkdir(exist_ok=True)

        #slide_reader.prepare_tiles(str(output_path) + "/" + file_path.stem)
################

In [ ]:
import os
import slideio
import numpy as np
from pathlib import Path
import cv2
import csv
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from PIL import Image
from itertools import product

# Path to the main folder containing sub-folders
main_folder = 'C:/Users/ga-steynlab-03/Demo_TB/wsi_svs_tiles'  # Replace with your main folder path

### Function to create the prediction model ###########################################
def create_model_from_weights(model_file):
    # The model is based on Inceptionv3 architecture, with changed classification layer
    RANDOM_SEED = 27
    model = keras.applications.InceptionV3(include_top=False, input_tensor=Input(shape=(224, 224, 3)))

    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1, seed=RANDOM_SEED)(x)
    predictions = Dense(2, activation='softmax')(x)

    model = Model(inputs=model.input, outputs=predictions)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.load_weights(model_file)

    return model

### Function to normalise images ######################################################
def read_normalized_image(file_name):
    # Check if file exists
    if not os.path.isfile(file_name):
        print(f"File '{file_name}' does not exist.")
        return None
    
    img = cv2.imread(file_name)
    
    # Check if image was successfully loaded
    if img is None:
        print(f"Failed to load image: {file_name}")
        return None
    
    # Changing the order of colour to keep the same ordering used while training
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Rescaling to network input resolution
    img = cv2.resize(img, (224, 224))
    # Standardization of pixel values
    mean = np.mean(img, axis=(0,1), keepdims=True)
    std = np.sqrt(((img - mean) ** 2).mean(axis=(0,1)))
    img = (img - mean) / (std + 0.000001)

    return img

### Function to create CSV file for a sub-folder########################################
def create_csv_for_subfolder(subfolder_path):
    # Extract sub-folder name from its path
    subfolder_name = os.path.basename(subfolder_path)
    
    # Define CSV file path for the sub-folder
    csv_file_path = os.path.join(subfolder_path, f"{subfolder_name}.csv")
    
    # Write to CSV file
    with open(csv_file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['File', 'Type', 'Score'])  # Header row   
        
    print(f"CSV file created for sub-folder '{subfolder_name}' at: {csv_file_path}")


def process_files_in_subfolder(subfolder_path, model):
    # Initialize list to store rows for CSV
    rows = [] 
    # Define CSV file path for the sub-folder
    subfolder_name = os.path.basename(subfolder_path)
    csv_file_path = os.path.join(subfolder_path, f"{subfolder_name}.csv")

    # Iterate through files in the sub-folder (assuming only .png files)
    for filename in os.listdir(subfolder_path):
        if filename.endswith('.png'):
            file_path = os.path.join(subfolder_path, filename)
            
            # Process the .png file here (e.g., read, analyze, etc.)
            img = read_normalized_image(str(file_path))
            org = cv2.imread(str(file_path))
            predicted = model.predict(np.expand_dims(img, axis=0))
            predicted_class = np.argmax(predicted)
            predicted_score = predicted[0][predicted_class]

            # Construct the line to be written in CSV
            line = "{file} is {type} with score {score}".format(file=filename,
                type="POSITIVE" if predicted_class == 1 else "NEGATIVE",
                score=predicted_score
            )
        
            # Append to the rows list
            rows.append([filename, "POSITIVE" if predicted_class == 1 else "NEGATIVE", predicted_score])
        else:
            print("No png image - process_files_in_subfolder.")
            
    # Append data to CSV file
    with open(csv_file_path, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerows(rows)
           

### Main Function
def main():
    #function call to create model
    model_file = "fold1_bestepoch4_bestacc0.9848_True.h5"
    model = create_model_from_weights(model_file=model_file)
    
    # Iterate through sub-folders in the main folder
    for subdir in os.listdir(main_folder):
        subfolder_path = os.path.join(main_folder, subdir)
        if os.path.isdir(subfolder_path):
            print(f"Processing sub-folder: {subfolder_path}")
            create_csv_for_subfolder(subfolder_path) ### Function call to create a csv file
            process_files_in_subfolder(subfolder_path, model) ### Function call to read and process the png files
        else:
            print("Incorrect path...")

if __name__ == "__main__":
    main()

Processing sub-folder: C:/Users/ga-steynlab-03/Demo_TB/wsi_svs_tiles\07-21114-E
CSV file created for sub-folder '07-21114-E' at: C:/Users/ga-steynlab-03/Demo_TB/wsi_svs_tiles\07-21114-E\07-21114-E.csv
No png image - process_files_in_subfolder.
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/ste

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1